# Octave Convolution (OctConv)

Bu notebook’ta **Octave Convolution (OctConv)** kavramını sıfırdan, **neden ortaya çıktığını**, **nasıl çalıştığını**, **hangi hiperparametrelerle kontrol edildiğini** ve **PyTorch ile nasıl uygulanacağını** adım adım inceleyeceğiz.

## Hedefler
- Feature map’lerdeki **yüksek frekans / düşük frekans** bilgisini anlamak
- OctConv’un **kanalları iki bantta (High/Low)** temsil etme fikrini kavramak
- `alpha` (α) parametresinin ne olduğunu ve pratik etkisini görmek
- OctConv’un içindeki dört yolu (H→H, H→L, L→H, L→L) netleştirmek
- PyTorch implementasyonu ile **çalışan** örnek kodlar elde etmek


## 1) Motivasyon: Neden OctConv?

Klasik konvolüsyonlarda giriş feature map’inin tamamı **aynı uzamsal çözünürlükte (H×W)** işlenir.

Fakat CNN feature map’leri şunu gösterir:
- Bazı kanallar **detay/kenar/tekstür** taşır → **yüksek frekans** (High-Freq)
- Bazı kanallar **genel şekil, geniş alan bağlamı** taşır → **düşük frekans** (Low-Freq)

Low-freq bilgi genellikle **daha düşük uzamsal çözünürlükte** saklanabilir.

OctConv fikri:
> Kanalları iki gruba ayır: **High (H)** ve **Low (L)**.
>
> Low kanalları **daha düşük çözünürlükte** tut (örn. H/2 × W/2).
>
> Böylece hesap yükü (FLOP) ve bellek hareketi azalır.

Bu fikir özellikle büyük backbone’larda FLOP düşürmek için değerlidir.


## 2) Frekans Mantığı: High vs Low

Frekans burada sinyal işleme anlamında sezgisel kullanılır:

- **High-Frequency (yüksek frekans):**
  - Keskin değişimler, kenarlar, ince detaylar
  - Uzamsal çözünürlüğe ihtiyaç duyar (H×W)

- **Low-Frequency (düşük frekans):**
  - Yavaş değişim, geniş bağlam, global şekil
  - Daha düşük çözünürlükte temsil edilebilir (H/2×W/2)

OctConv, bu ayrımı **kanal düzeyinde** yapar.


## 3) OctConv’un Ana Parametresi: α (alpha)

OctConv’da kanalların ne kadarı Low band’a gidecek bunu **α** belirler.

- Giriş kanalı sayısı: `C_in`
- Çıkış kanalı sayısı: `C_out`

Low bant kanal sayıları:
- `C_in_L = α_in * C_in`
- `C_out_L = α_out * C_out`

High bant kanal sayıları:
- `C_in_H = C_in - C_in_L`
- `C_out_H = C_out - C_out_L`

Tipik kullanım:
- `α` genelde 0.25 veya 0.5 gibi seçilir.

**α arttıkça:**
- Low frekans kanalları artar → daha çok hesap düşük çözünürlükte yapılır
- FLOP azalabilir ama detay taşıyan kapasite düşebilir


## 4) OctConv İç Yapısı: 4 Akış (H→H, H→L, L→H, L→L)

OctConv, girişte iki bant alır: `(X_H, X_L)`.
Çıkışta da iki bant üretir: `(Y_H, Y_L)`.

### 4 yol
1. **H → H:** High’tan High’a normal conv (aynı çözünürlük)
2. **H → L:** High’tan Low’a bilgi aktarımı
   - Önce downsample (örn. avgpool), sonra conv
3. **L → H:** Low’tan High’a bilgi aktarımı
   - Conv, sonra upsample (örn. nearest/bilinear)
4. **L → L:** Low’tan Low’a conv (düşük çözünürlükte)

Sezgisel formül:

$Y_H = f_{H\to H}(X_H) + \mathrm{Up}(f_{L\to H}(X_L))$

$Y_L = f_{L\to L}(X_L) + f_{H\to L}(\mathrm{Down}(X_H))$

Burada `f`'ler konvolüsyon işlemleridir.


## 5) Pratikte OctConv Nasıl Kullanılır?

OctConv genelde 3 senaryoda kullanılır:

### A) İlk OctConv (girişte tek bant var)
- Giriş sadece High’tır: `X_L` yok.
- `X_H`'ın bir kısmı downsample edilerek Low bant oluşturulur.

### B) Ara OctConv (girişte iki bant var)
- Giriş `(X_H, X_L)` olarak gelir.
- Çıkış yine `(Y_H, Y_L)` üretir.

### C) Son OctConv (çıkışta tek bant istersin)
- Çıkışta Low bantı High’a upsample edip birleştirirsin.
- Böylece klasik katmanlara geri dönersin.

Bu notebook’ta, en anlaşılır olan **Ara OctConv** yapısını kodlayacağız.


## 6) PyTorch: OctConv Implementasyonu (Çalışan)

Aşağıdaki sınıf:
- Girdi: `(x_h, x_l)` (Low yoksa `x_l=None` verilebilir)
- Çıktı: `(y_h, y_l)`

Basit ve öğretici olması için:
- Downsample: `AvgPool2d(2)`
- Upsample: `F.interpolate(scale_factor=2, mode='nearest')`


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class OctaveConv2d(nn.Module):
    def __init__(self, cin, cout, kernel_size=3, stride=1, padding=1,
                 alpha_in=0.5, alpha_out=0.5, bias=False):
        super().__init__()

        assert 0.0 <= alpha_in <= 1.0
        assert 0.0 <= alpha_out <= 1.0

        self.alpha_in = alpha_in
        self.alpha_out = alpha_out
        self.stride = stride

        cin_l = int(round(cin * alpha_in))
        cin_h = cin - cin_l

        cout_l = int(round(cout * alpha_out))
        cout_h = cout - cout_l

        self.cin_h, self.cin_l = cin_h, cin_l
        self.cout_h, self.cout_l = cout_h, cout_l

        self.down = nn.AvgPool2d(kernel_size=2, stride=2)

        self.hh = nn.Conv2d(cin_h, cout_h, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_h > 0 and cout_h > 0) else None
        self.hl = nn.Conv2d(cin_h, cout_l, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_h > 0 and cout_l > 0) else None
        self.lh = nn.Conv2d(cin_l, cout_h, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_l > 0 and cout_h > 0) else None
        self.ll = nn.Conv2d(cin_l, cout_l, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_l > 0 and cout_l > 0) else None

    def forward(self, x_h, x_l=None):
        y_h = 0
        y_l = 0

        if self.hh is not None and x_h is not None:
            y_h = y_h + self.hh(x_h)

        if self.hl is not None and x_h is not None:
            x_h_down = self.down(x_h)
            y_l = y_l + self.hl(x_h_down)

        if self.ll is not None and x_l is not None:
            y_l = y_l + self.ll(x_l)

        if self.lh is not None and x_l is not None:
            y_lh = self.lh(x_l)
            y_lh_up = F.interpolate(y_lh, scale_factor=2, mode='nearest')
            y_h = y_h + y_lh_up

        if isinstance(y_h, int):
            y_h = None
        if isinstance(y_l, int):
            y_l = None

        return y_h, y_l


# Hızlı test
B = 2
cin, cout = 64, 128
alpha = 0.5
cin_l = int(round(cin * alpha))
cin_h = cin - cin_l

x_h = torch.randn(B, cin_h, 32, 32)
x_l = torch.randn(B, cin_l, 16, 16)

oct = OctaveConv2d(cin, cout, alpha_in=alpha, alpha_out=alpha)
y_h, y_l = oct(x_h, x_l)
print('x_h:', x_h.shape, 'x_l:', x_l.shape)
print('y_h:', None if y_h is None else y_h.shape, 'y_l:', None if y_l is None else y_l.shape)


x_h: torch.Size([2, 32, 32, 32]) x_l: torch.Size([2, 32, 16, 16])
y_h: torch.Size([2, 64, 32, 32]) y_l: torch.Size([2, 64, 16, 16])


## 7) OctConv Bloğu: OctConv + BN + Activation

Backbone’da kullanmak için genelde BN/Activation eklenir.
Aşağıdaki blok, High ve Low bantlara ayrı BN uygular.


In [2]:
class OctaveConvBlock(nn.Module):
    def __init__(self, cin, cout, alpha_in=0.5, alpha_out=0.5,
                 kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.oct = OctaveConv2d(cin, cout, kernel_size, stride, padding, alpha_in, alpha_out, bias=False)

        self.bn_h = nn.BatchNorm2d(self.oct.cout_h) if self.oct.cout_h > 0 else None
        self.bn_l = nn.BatchNorm2d(self.oct.cout_l) if self.oct.cout_l > 0 else None
        self.act = nn.ReLU(inplace=True)

    def forward(self, x_h, x_l=None):
        y_h, y_l = self.oct(x_h, x_l)
        if y_h is not None and self.bn_h is not None:
            y_h = self.act(self.bn_h(y_h))
        if y_l is not None and self.bn_l is not None:
            y_l = self.act(self.bn_l(y_l))
        return y_h, y_l


# Test
block = OctaveConvBlock(64, 128, alpha_in=0.5, alpha_out=0.5)
y_h, y_l = block(x_h, x_l)
print('block y_h:', y_h.shape if y_h is not None else None)
print('block y_l:', y_l.shape if y_l is not None else None)


block y_h: torch.Size([2, 64, 32, 32])
block y_l: torch.Size([2, 64, 16, 16])


## 8) Mini Model Örneği: OctConv Kullanan Basit CNN

Bu model:
- Stem ile 64 kanal üretir
- High/Low bantlara split eder
- 2 OctConv blok uygular
- Low’u upsample edip High ile birleştirir
- GlobalAvgPool + Linear ile sınıflandırır


In [3]:
class MiniOctNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, alpha=0.5):
        super().__init__()
        self.alpha = alpha

        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )

        self.b1 = OctaveConvBlock(64, 128, alpha_in=alpha, alpha_out=alpha)
        self.b2 = OctaveConvBlock(128, 128, alpha_in=alpha, alpha_out=alpha)

        self.down = nn.AvgPool2d(2)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def split_hl(self, x):
        B, C, H, W = x.shape
        c_l = int(round(C * self.alpha))
        c_h = C - c_l
        x_h = x[:, :c_h]
        x_l = self.down(x[:, c_h:]) if c_l > 0 else None
        return x_h, x_l

    def merge_hl(self, x_h, x_l):
        if x_l is None:
            return x_h
        x_l_up = F.interpolate(x_l, scale_factor=2, mode='nearest')
        return torch.cat([x_h, x_l_up], dim=1)

    def forward(self, x):
        x = self.stem(x)
        x_h, x_l = self.split_hl(x)
        x_h, x_l = self.b1(x_h, x_l)
        x_h, x_l = self.b2(x_h, x_l)
        x = self.merge_hl(x_h, x_l)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


# Test
m = MiniOctNet(in_channels=3, num_classes=10, alpha=0.5)
dummy = torch.randn(4, 3, 32, 32)
out = m(dummy)
print('out:', out.shape)


out: torch.Size([4, 10])


## 9) Parametre Karşılaştırması (Basit)

OctConv’un parametreleri dört yolun toplamıdır.
Bu toplam çoğu zaman klasik conv ile aynı mertebededir.

Asıl avantaj **FLOP** tarafında gelir çünkü Low bant konvolüsyonları daha düşük çözünürlükte çalışır.


In [4]:
def conv_params(cin, cout, k):
    return cin * cout * k * k

def octconv_params(cin, cout, k, alpha_in=0.5, alpha_out=0.5):
    cin_l = int(round(cin * alpha_in)); cin_h = cin - cin_l
    cout_l = int(round(cout * alpha_out)); cout_h = cout - cout_l
    return (cin_h*cout_h + cin_h*cout_l + cin_l*cout_h + cin_l*cout_l) * k*k

cin, cout, k, alpha = 64, 128, 3, 0.5
print('Conv params:', conv_params(cin, cout, k))
print('OctConv params:', octconv_params(cin, cout, k, alpha, alpha))


Conv params: 73728
OctConv params: 73728


## 10) Ne Zaman Mantıklı?

- Erken/orta aşamada çözünürlük yüksekken (H×W büyük) FLOP düşürmek için iyi olabilir.
- Çok büyük α seçmek detay kaybı riskini artırır.
- İki bant yönetimi kod karmaşıklığını artırır.

Yine de OctConv, **"feature map’in her kanalı aynı çözünürlükte tutulmak zorunda değil"** fikrini öğretmesi açısından çok değerlidir.


## 11) Özet

- OctConv, kanalları **High/Low** frekans olarak ayırır.
- Low bant düşük çözünürlükte işlenir → **FLOP azalır**.
- 4 akış: H→H, H→L, L→H, L→L.
- `α` low kanal oranını belirler.
- Pratikte girişte split, ortada OctConv, çıkışta merge yapılır.
